In [1]:
import h5py
import sys
sys.path.append('/Users/zhengj10/Python/Modules')
import os
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
import yaml
import pandas as pd
import PatcherBot as pb
from glob import glob
import seaborn as sns


In [2]:
imgPath = '/Volumes/genie/BenArthur/patch-screen/bot'
rawPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw'
sumPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/summary'

os.path.exists(rawPath)


True

In [3]:
df = pd.read_pickle(sumPath+'/Summary_pos_to_neg_VC.pkl')
df.shape

(330, 30)

In [4]:
# df = df.drop(df.index[np.arange(25)])
df

,Date,Plate,Well,Cell,Mutation,Transfected_date,Leak,Ra,dFF,F0,...,tau_on_f_double,tau_off_f_double,tau_on_s_double,tau_off_s_double,time_on_10-90,time_off_10-90,Leak_min,Ra_max,peak_dFF,Objective
0,2021.03.03_000,plate_000,well_2,patch_attempt_0000_000,421.1,210223,"[-8.51146335323723, -11.243772409311036, -19.8...","[-1057.365960924749, -764.9392708633397, 22.60...","[-0.14526184007841117, -0.13039366071182862, -...","[154.9519814756523, 153.82582677889638, 152.77...",...,"[4.05850660687916, 4.621801229291567, 6.029508...","[2.552167342182723, 3.0706327088517518, 2.1826...","[0.4267672185280103, 4.629006137721192, 6.0352...","[338.70305765574324, 52261.44504437979, 718.21...","[7.782478874019432, 10.15905319990594, 13.2512...","[8.947719009634284, 6.7517492375448, 5.2234200...",-30.691261,27.651168,"[-0.16507148189795556, -0.15830654624872487, -...",40x oil GENIE
1,2021.03.03_000,plate_000,well_2,patch_attempt_0003_000,421.1,210223,"[-147.85085551011414, -110.8364727143926, -129...","[-1614.363575533173, -676.82899220251, 19.8547...","[-0.14410990459331446, -0.13609067412031509, -...","[165.31300964702248, 164.97381968025476, 161.5...",...,"[0.4548669266003282, 0.6525479868127556, 0.780...","[3.598058127122556, 1.9690762141661755, 1.8760...","[0.4541163594439905, 0.6819302826389929, 0.767...","[25.095314176668825, 1.917025752929548, 1.8726...","[nan, nan, nan, nan, nan, 5.187198196080203, 5...","[6.498641656564359, 4.3033488648750335, 4.1212...",-203.530114,19.892649,"[-0.2477342052135697, -0.23731019355486327, -0...",40x oil GENIE
2,2021.03.03_000,plate_000,well_2,patch_attempt_0004_000,421.1,210223,"[-397.816589224832, -409.7027167633756, -452.5...","[-934.1572600380731, -594.8388183232978, 19.46...","[-0.15919929154447982, -0.14500278783245382, -...","[141.08997519518655, 140.42054814785416, 139.1...",...,"[0.8752452645976473, 1.0529383560611316, 0.745...","[1.8570281425892907, 1.883731087014039, 3.0057...","[0.8835280408491996, 1.0695482922809778, 0.704...","[1.377806995308405, 1.9191511339477574, 2.9740...","[nan, nan, nan, nan, nan, 5.3698919651374695, ...","[4.0680605481970815, 4.154835356999229, 6.5903...",-656.624857,19.468605,"[-0.2280239587623224, -0.19460127609269418, -0...",40x oil GENIE
3,2021.03.03_000,plate_000,well_2,patch_attempt_0005_000,421.1,210223,"[-80.79750022380867, -90.01445331808492, -87.4...","[-990.9833494549036, -787.3084931624882, 21.42...","[-0.15521434691726646, -0.15565164142640084, -...","[246.1049289831738, 244.052892177803, 242.3198...",...,"[0.390108466417082, 0.643269775577479, 0.96978...","[1.729817845003191, 1.8184609235121538, 2.0329...","[0.3902293258196524, 0.6293954925684992, 1.001...","[1.604310634346249, 1.7836541465098967, 2.1069...","[nan, nan, nan, nan, nan, 4.447202685168179, 0...","[3.742462115287026, 3.979903165355381, 4.48663...",-167.043379,22.781844,"[-0.30144233609463744, -0.27303197451385064, -...",40x oil GENIE
4,2021.03.03_000,plate_000,well_2,patch_attempt_0009_000,421.1,210223,"[-21.855871428249863, -37.37691393853611, -38....","[-913.2990455733028, -704.4073729178376, 7.556...","[-0.19202265117289682, -0.183520186021923, -0....","[213.5878493573361, 211.1675211508787, 208.609...",...,"[0.7176699394289654, 0.9713875866008165, 1.268...","[2.442859426598716, 2.236323629495338, 2.46128...","[0.0005584134281668861, 0.9800234421979227, 1....","[2.3813339513539127, 88256.1853004978, 110442....","[nan, nan, nan, nan, nan, 7.311644915460022, 9...","[5.345403031942396, 4.915514641842719, 5.40997...",-51.636494,7.556346,"[-0.2700482285135941, -0.2349602862865643, -0....",40x oil GENIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2021.03.26_000,plate_004,well_1,patch_attempt_0043_000,421.6004,210316,"[-37.94002573762585, -93.55126468627827, -107....","[-888.4441334111231, -796.5426447247947, 9.775...","[-0.18682281863474995, -0.16913503398966878, -...","[258.4258054704354, 251.2902373436457, 244.888...",...,"[1.182807897381437, 

### Load params from .mat file

In [5]:
# datePath = pb.getPathInfo(imgPath)
datePath = ['2021.04.16_000']
# datePath = datePath[10:]
# datePath = ['2020.10.28_001','2020.10.28_001-varnam']

datePath


['2021.04.16_000']

In [6]:
for date in datePath:
    print(date[:14])

2021.04.16_000


In [7]:
data_all={}

for param in(df.columns):
    data_all[param] = []
    
for date in datePath:
    platePath = pb.getPathInfo(os.path.join(imgPath, date))
    
    for plate in platePath:
    
        idxFile = os.path.join(rawPath, date[:14], plate, '.index')
        a = yaml.load(open(idxFile), Loader=yaml.FullLoader)
        
#         if date == '2021.03.17_000' and plate=='plate_000':
        
#         mutation = a['.']['mutation_2']
#         else:
        mutation = a['.']['mutation_1']
            
        note = a['.']['notes'][0:6]

        cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
        for cell in(cellPath):
            matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
            if len(matFiles) < 9:
                print('bad trial: ' + '/' + date + '/' + plate + '/' + cell)
                continue
#             print(len(matFiles))
            print(os.path.join(rawPath, date, plate))
    
            f = h5py.File(matFiles[0],'r')
            pix_index = f['params']['pix_index'][()]
            if type(pix_index) == h5py._hl.base.Empty:
                print('No responsive pixels')
                continue
                
            data_all['Date'].append(date[:14])
            data_all['Plate'].append(plate)
            data_all['Well'].append('well_1')
            data_all['Cell'].append(cell)
            data_all['Mutation'].append(mutation)
            data_all['Transfected_date'].append(note[:6])
            leaks = []
            Ra_s = [] # List of Ra
            F0 = []

            dff=[]
            peak_dff = []
            amp_on = []
            amp_off = []
            tau_on = []
            tau_off = []

            amp_on_fast = []
            amp_on_slow = []
            amp_off_fast = []
            amp_off_slow = []
            tau_on_fast = []
            tau_on_slow = []
            tau_off_fast = []
            tau_off_slow = []
            time_on = []
            time_off = []
            bleach_x = []
            bleach = []

            for i in range(9):
                
                
#                 if date == '2021.03.17_000' and plate=='plate_000':

                ivFile = os.path.join(rawPath,date[:14],plate,cell,'camera_clamp1_000',str(i).zfill(3),'Clamp1.ma')
#                 else:
#                     ivFile = os.path.join(rawPath,date,plate,cell,'camera_clamp1_000',str(i).zfill(3),'Clamp1.ma')
                leak = pb.get_leak(ivFile)
                Ra = pb.get_Ra(ivFile,i)
                
                matFile = os.path.join(imgPath, date, plate, cell) + '/' + str(i).zfill(3) + '.mat'
                if not os.path.exists(matFile):
                    print(str(i).zfill(3)+'.mat not existing')
                    continue
                data = pb.get_data(matFile)
                f0_single = data['f0']

                leaks.append(leak)
                Ra_s.append(Ra)
                F0.append(f0_single)

                amp_on_single = data['amp_on'][0]
                amp_off_single = data['amp_off'][0]
                tau_on_single = data['tau_on'][0]
                tau_off_single = data['tau_off'][0]
                bleach_single = data['bleach']
                bleach_x_single = data['bleach_x']
                
                

                amp_on.append(amp_on_single)
                amp_off.append(amp_off_single)
                tau_on.append(tau_on_single)
                tau_off.append(tau_off_single)
                bleach_x.append(bleach_x_single)
                bleach.append(bleach_single)

                trace_double = data['dff_corrected_double']                # for double fit
                dff_double = pb.get_dff(trace_double)                
                peak_dff_double = pb.get_peak_dff(trace_double,i)
                
                amp_on_fast_single = data['amp_on_fast'][0]                
                amp_on_slow_single = data['amp_on_slow'][0]
                amp_off_fast_single = data['amp_off_fast'][0]
                amp_off_slow_single = data['amp_off_slow'][0]
                tau_on_fast_single = data['tau_on_fast'][0]
                tau_on_slow_single = data['tau_on_slow'][0]
                tau_off_fast_single = data['tau_off_fast'][0]
                tau_off_slow_single = data['tau_off_slow'][0]
                time_on_double = data['time_on'][0]
                if time_on_double == 0:
                    print('time_on: ' + date + '/' + plate + '/' + cell + '/' + str(i).zfill(3) + '.mat')
                time_off_double = data['time_off'][0]
                if time_off_double == 0:
                    print('time_off: ' + date + '/' + plate + '/' + cell + '/' + str(i).zfill(3) + '.mat')

                dff.append(dff_double)
                peak_dff.append(peak_dff_double)

                amp_on_fast.append(amp_on_fast_single)
                amp_on_slow.append(amp_on_slow_single)
                amp_off_fast.append(amp_off_fast_single)
                amp_off_slow.append(amp_off_slow_single)
                tau_on_fast.append(tau_on_fast_single)
                tau_on_slow.append(tau_on_slow_single)
                tau_off_fast.append(tau_off_fast_single)
                tau_off_slow.append(tau_off_slow_single)
                time_on.append(time_on_double)
                time_off.append(time_off_double)

            data_all['F0'].append(F0)
            data_all['Leak'].append(leaks)
            data_all['Ra'].append(Ra_s)
            data_all['Leak_min'].append(np.min(leaks))
            data_all['Ra_max'].append(np.max(Ra_s))

            data_all['dFF'].append(dff)
            data_all['peak_dFF'].append(peak_dff)
            
            
            data_all['amp_on_single'].append(amp_on)
            data_all['amp_off_single'].append(amp_off)
            data_all['tau_on_single'].append(tau_on)
            data_all['tau_off_single'].append(tau_off)
            data_all['time_on_10-90'].append(time_on)
            data_all['time_off_10-90'].append(time_off)
            data_all['bleach_x'].append(bleach_x)
            data_all['bleach'].append(bleach)

            data_all['amp_on_f_double'].append(amp_on_fast)
            data_all['amp_on_s_double'].append(amp_on_slow)
            data_all['amp_off_f_double'].append(amp_off_fast)
            data_all['amp_off_s_double'].append(amp_off_slow)
            data_all['tau_on_f_double'].append(tau_on_fast)
            data_all['tau_on_s_double'].append(tau_on_slow)
            data_all['tau_off_f_double'].append(tau_off_fast)
            data_all['tau_off_s_double'].append(tau_off_slow)
            data_all['Objective'].append('40x oil GENIE')
                
print('Done!')


/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
No responsive pixels
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
No responsive pixels
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_000
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_001
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_001
/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2021.04.16_000/plate_

In [8]:
for k,v in data_all.items():
#     data_all[k] = data_all[k][:22]
    print(k,len(v))
# data_all['Date'][[503]]

Date 27
Plate 27
Well 27
Cell 27
Mutation 27
Transfected_date 27
Leak 27
Ra 27
dFF 27
F0 27
amp_on_single 27
amp_off_single 27
tau_on_single 27
tau_off_single 27
bleach_x 27
bleach 27
amp_on_f_double 27
amp_on_s_double 27
amp_off_f_double 27
amp_off_s_double 27
tau_on_f_double 27
tau_off_f_double 27
tau_on_s_double 27
tau_off_s_double 27
time_on_10-90 27
time_off_10-90 27
Leak_min 27
Ra_max 27
peak_dFF 27
Objective 27


In [10]:
df.shape

(330, 30)

In [9]:
df_new = pd.DataFrame(data_all, columns = list(df)) # columns specification keeps the order of columns; index specification makes sure dict keys are the columns 
df_new.shape

(27, 30)

In [11]:
df = df_new.append(df, ignore_index = True)
df.index = np.arange(len(df))
df.shape

(357, 30)

In [13]:
# df.to_pickle(sumPath+'/Summary_VC_all.pkl')
# df.to_excel(sumPath + '/Summary_VC_all.xlsx')
df.to_pickle(sumPath+'/Summary_pos_to_neg_VC_p_val_10-50.pkl')

In [ ]:
# df = df.drop(df.index[len(df)-1]) # Drop the last row
# df


In [ ]:
# Check the size for each column is the same
# i = 0
# for k, v in data_all.items():
#     print(k, len(v))
#     i+=1


In [ ]:
matPath = imgPath+'/2021.03.18_000/plate_001/patch_attempt_0013_000/'

In [ ]:
for trial in range(9):
    print(trial)
    matFile = matPath+str(trial).zfill(3)+'.mat'
    f = h5py.File(matFile,'r')
    pix_index = f['params']['pix_index'][()]
    print(len(pix_index))
    index_list = [f[pix_index[i]][()][0] for i in range(len(pix_index))]
    index_list = np.array(index_list)
    index_list -= 1 
    img_F0 = f['params']['img_F0'][()]
    img_F0 = img_F0.flatten()

    # f0 = np.mean(img_F0[index_list])
    # data['f0'] = f0

    d = f['params']['fit_pix_double'] # for double fit
    

In [ ]:
len(d)

In [ ]:
st = d[0]
obj = f[st]

In [ ]:
a = obj['bleach_bkg_norm_ave_trace'][()]

In [ ]:
bleach = obj['bkg_norm_ave_bleach_parameters'][()][1]
bleach_x = obj['bkg_norm_ave_bleach_parameters'][()][0]

In [ ]:
bleach

In [ ]:
obj['bkg_norm_ave_bleach_parameters'][()]

In [ ]:
plt.plot(a)